In [1]:
# imports
import pandas as pd
import os
import matplotlib.pyplot as plt
import re

# set filepath to data source
filepath = "games.csv"


In [2]:
# load data into dataframe
games_df = pd.read_csv(filepath)
games_df.head(1)

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5


In [3]:
# split opening_name into opening and variation
opening_variation_s = games_df["opening_name"].str.split(": ", n=1, expand = True)
games_df["opening"] = opening_variation_s[0]
games_df["variation"] = opening_variation_s[1]

# cleanup opening so similar names correlate properly 
# (eg. "Queens Pawn" and "Queens Pawn Game")

# map for splitting the opening_varient into (opening, varient)
opening_map = {}
# collection for names already tested
openings = []
for opening in games_df["opening"].value_counts().index:
  # 
  if ~(opening in openings):
    for op in openings:
      if opening == op:
        continue
      else if (op.startswith(opening)):
        openings.remove(op)
        openings.append(opening)
        opening_map[op] = (opening, op[len(opening):])
# TODO: FINISH
    # split(opening) into opening and variation
    # map update opening/variation on rows with that pattern
  


In [4]:

games_df["opening"].value_counts().index

Index(['Sicilian Defense', 'French Defense', 'Queen's Pawn Game',
       'Italian Game', 'King's Pawn Game', 'Ruy Lopez', 'English Opening',
       'Scandinavian Defense', 'Caro-Kann Defense', 'Scotch Game',
       ...
       'King's Gambit |  Falkbeer Countergambit |  Miles Gambit',
       'King's Indian |  Orthodox |  7...Nbd7 8.Re1',
       'Danish Gambit Accepted |  Schlechter Defense', 'Doery Defense',
       'Benko Gambit Declined |  Main Line',
       'King's Gambit Accepted |  Rosentreter Gambit',
       'Benko Gambit Declined |  Quiet Line', 'Pterodactyl Defense',
       'Valencia Opening', 'Benko Gambit Accepted |  Pawn Return Variation'],
      dtype='object', length=227)

In [11]:
# filter out bad data (resignations, unranked games)
# bad cases:  player rating missmatch, <10 turns resignation, no pieces lost resignation


,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply,opening,variation
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5,Slav Defense,Exchange Variation
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4,Nimzowitsch Defense,Kennedy Variation
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3,King's Pawn Game,Leonardis Variation
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3,Queen's Pawn Game,Zukertort Variation
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5,Philidor Defense,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,EfqH7VVH,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,A80,Dutch Defense,2,Dutch Defense,None
20054,WSJDhbPl,True,1.499698e+12,1.499699e+12,82,mate,black,10+0,jamboger,1233,farrukhasomiddinov,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...,A41,Queen's Pawn,2,Queen's Pawn,None
20055,yrAas0Kj,True,1.499698e+12,1.499698e+12,35,mate,white,10+0,jamboger,1219,schaaksmurf3,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...,D00,Queen's Pawn Game: Mason Attack,3,Queen's Pawn Game,Mason Attack
20056,b0v4tRyF,True,1.499696e+12,1.499697e+12,109,resign,white,10+0,marcodisogno,1360,jamboger,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...,B07,Pirc Defense,4,Pirc Defense,None


In [6]:
# get general summary data 

## overall wins for white/black
## counts of openings

## graphs to visualize these

In [7]:
# grouping by opening
# find the win rates for each opening/varient

# graph these

In [8]:
# find the number of pieces taken by white/black each game

In [9]:
# grouping by opening, winner
# find out how many pieces the winner had taken 
#   vs the count of pieces the opp. had taken

# graph this seperately for white as winner and black as winner

In [10]:
# graph % of times the lower rated player wins per opening per color

In [ ]:
# repeat above work, but with the data binned by ranges of player ratings